In [85]:
import os
import sys 
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K

from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import plotly
import plotly.graph_objects as go
import plotly.express as px
import kaleido ##pip install -U kaleido ##to save a plotly fig
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score,cross_val_predict

In [86]:
df1 = pd.read_csv('Stable_Data_CSV.csv')
df2 = pd.read_csv('6wUsingBL.csv')
df_stable = df2[df2.set_index(['participant_id']).index.isin(df1.set_index(['participant_id']).index)]
df_stable_US = df_stable.loc[(df_stable['arm'] == 1)]
df_stable_WN = df_stable.loc[(df_stable['arm'] == 2)]
print(len(df_stable_US))
print(len(df_stable_WN))
df_stable_US=(df_stable_US[['c_3','sl_1','a_2','q_1','tfi_total.1','tfi_total.2','tfi_total.3']])
df_stable_WN=(df_stable_WN[['sl_2','r_1','e_1','tfi_total.1','tfi_total.2','tfi_total.3']])

31
30


In [87]:
def data_prep(df):
    sc_X = StandardScaler()
    sc_y = StandardScaler()
    
    X=df.drop(['tfi_total.2','tfi_total.3'],axis=1)
    y=df[['tfi_total.2']]
    
    #scale x
    x_scaler=sc_X.fit(X)
    X=x_scaler.transform(X)
    #scale y
    y_scaler=sc_y.fit(y)
    y=y_scaler.transform(y)
    
    """
    Reshape rule:
    tensor of shape (batch size, sequence length, features), 
    where sequence length is the number of time steps and features is each input timeseries.
    """
    X = X.reshape((X.shape[0], X.shape[1], 1))
    print(X.shape,y.shape)
    
    return X,y,x_scaler,y_scaler

In [88]:

"""
Include residual connections, layer normalization, and dropout.
The resulting layer can be stacked multiple times.
The projection layers are implemented through `keras.layers.Conv1D`.
"""

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    K.clear_session()
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    """    
    can stack multiple of the transformer_encoder blocks and 
    can also proceed to add the final Multi-Layer Perceptron regression head.
    """
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    
    """
    a pooling layer is used to to reduce the output tensor of the TransformerEncoder 
    part of our model down to a vector of features for each data point in the current batch.
    """
    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x) 
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)
    return keras.Model(inputs, outputs)


In [113]:
## Train and evaluate


def prediction(x_train,y_train,y_scaler):
    input_shape = x_train.shape[1:]

    model = build_model(
        input_shape,
        head_size=5, # Embedding size for each token #key_dim
        num_heads=4, # Number of attention heads
        ff_dim=5, # Hidden layer size in feed forward network inside transformer
        num_transformer_blocks=4,
        mlp_units=[128],
        mlp_dropout=0.4,
        dropout=0.25,
    )

    def error_in_tfi(y_true,y_pred): 
        y=y_true.numpy()
        yhat=y_pred.numpy()
        y=np.reshape(y, (1,-1))
        yhat=np.reshape(yhat, (1,-1))
        y=y_scaler.inverse_transform(y)
        yhat=y_scaler.inverse_transform(yhat)
        y=tf.convert_to_tensor(y, dtype=tf.float32)
        yhat=tf.convert_to_tensor(yhat, dtype=tf.float32)
        return K.mean(abs(y - yhat), axis=-1)  #K.mean(square(y_true - y_pred), axis=-1)


    model.compile(
        loss="mse",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        #metrics=[keras.metrics.MeanAbsoluteError()],
        run_eagerly=True,
        metrics=[error_in_tfi],
    )
    model.summary()

    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

    history=model.fit(
        x_train,
        y_train,
        validation_split=0.2,
        epochs=200,
        batch_size=4,
        callbacks=callbacks,
    )
    
    return model,history
    #model.evaluate(x_test, y_test, verbose=1)


In [114]:
error_scores_US=[]
error_scores_WN=[]
kfold = KFold(n_splits=5)

def cross_val(df,error_scores):
    X,y,x_scaler,y_scaler=data_prep(df)
    for train_index, test_index in kfold.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
        model,history=prediction(x_train,y_train,y_scaler)
        val_mse,val_mae=model.evaluate(x_test,y_test) #evaluating using unseen data
        error_scores.append(val_mae)
    return history,x_scaler,y_scaler,error_scores

US_history,US_x_scaler,US_y_scaler,US_error_scores=cross_val(df_stable_US,error_scores_US)
WN_history,WN_x_scaler,WN_y_scaler,WN_error_scores=cross_val(df_stable_WN,error_scores_WN)

(31, 5, 1) (31, 1)
TRAIN: [ 7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30] TEST: [0 1 2 3 4 5 6]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 1)]       0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 5, 1)        7169        ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 5, 1)         0           ['multi_head_attention[0][0]']   
                                                                       

 tf.__operators__.add_6 (TFOpLa  (None, 5, 1)        0           ['layer_normalization_6[0][0]',  
 mbda)                                                            'tf.__operators__.add_5[0][0]'] 
                                                                                                  
 conv1d_6 (Conv1D)              (None, 5, 4)         8           ['tf.__operators__.add_6[0][0]'] 
                                                                                                  
 dropout_7 (Dropout)            (None, 5, 4)         0           ['conv1d_6[0][0]']               
                                                                                                  
 conv1d_7 (Conv1D)              (None, 5, 1)         5           ['dropout_7[0][0]']              
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 5, 1)        2           ['conv1d_7[0][0]']               
 rmalizati

5/5 [==============================] - 1s 138ms/step - loss: 0.9230 - error_in_tfi: 16.2260 - val_loss: 0.4300 - val_error_in_tfi: 7.8158
Epoch 38/200
5/5 [==============================] - 1s 144ms/step - loss: 0.9691 - error_in_tfi: 17.0249 - val_loss: 0.4298 - val_error_in_tfi: 7.8406
Epoch 39/200
5/5 [==============================] - 1s 151ms/step - loss: 1.0082 - error_in_tfi: 16.5597 - val_loss: 0.4295 - val_error_in_tfi: 7.8608
Epoch 40/200
5/5 [==============================] - 1s 141ms/step - loss: 0.8376 - error_in_tfi: 15.5718 - val_loss: 0.4293 - val_error_in_tfi: 7.8887
Epoch 41/200
5/5 [==============================] - 1s 141ms/step - loss: 0.9610 - error_in_tfi: 15.4993 - val_loss: 0.4290 - val_error_in_tfi: 7.8956
Epoch 42/200
5/5 [==============================] - 1s 138ms/step - loss: 1.0034 - error_in_tfi: 16.6372 - val_loss: 0.4287 - val_error_in_tfi: 7.9042
Epoch 43/200
5/5 [==============================] - 1s 145ms/step - loss: 0.7652 - error_in_tfi: 15.2327 - 

5/5 [==============================] - 1s 139ms/step - loss: 0.9428 - error_in_tfi: 15.5899 - val_loss: 0.4102 - val_error_in_tfi: 8.2494
Epoch 92/200
5/5 [==============================] - 1s 141ms/step - loss: 0.8558 - error_in_tfi: 15.7231 - val_loss: 0.4096 - val_error_in_tfi: 8.2474
Epoch 93/200
5/5 [==============================] - 1s 138ms/step - loss: 0.7737 - error_in_tfi: 14.3202 - val_loss: 0.4092 - val_error_in_tfi: 8.2500
Epoch 94/200
5/5 [==============================] - 1s 139ms/step - loss: 0.8592 - error_in_tfi: 15.0258 - val_loss: 0.4091 - val_error_in_tfi: 8.2742
Epoch 95/200
5/5 [==============================] - 1s 139ms/step - loss: 0.8468 - error_in_tfi: 14.9791 - val_loss: 0.4086 - val_error_in_tfi: 8.2640
Epoch 96/200
5/5 [==============================] - 1s 138ms/step - loss: 0.7435 - error_in_tfi: 14.3309 - val_loss: 0.4085 - val_error_in_tfi: 8.2781
Epoch 97/200
5/5 [==============================] - 1s 137ms/step - loss: 0.7195 - error_in_tfi: 13.4868 - 

5/5 [==============================] - 1s 137ms/step - loss: 0.6933 - error_in_tfi: 13.1459 - val_loss: 0.3906 - val_error_in_tfi: 8.4910
Epoch 146/200
5/5 [==============================] - 1s 139ms/step - loss: 0.7523 - error_in_tfi: 14.4102 - val_loss: 0.3904 - val_error_in_tfi: 8.5045
Epoch 147/200
5/5 [==============================] - 1s 139ms/step - loss: 0.6890 - error_in_tfi: 12.9019 - val_loss: 0.3902 - val_error_in_tfi: 8.5058
Epoch 148/200
5/5 [==============================] - 1s 139ms/step - loss: 0.7219 - error_in_tfi: 13.8036 - val_loss: 0.3901 - val_error_in_tfi: 8.5145
Epoch 149/200
5/5 [==============================] - 1s 139ms/step - loss: 0.8036 - error_in_tfi: 14.3846 - val_loss: 0.3896 - val_error_in_tfi: 8.5017
Epoch 150/200
5/5 [==============================] - 1s 139ms/step - loss: 0.7491 - error_in_tfi: 14.2823 - val_loss: 0.3892 - val_error_in_tfi: 8.5014
Epoch 151/200
5/5 [==============================] - 1s 139ms/step - loss: 0.7872 - error_in_tfi: 14.4

...

Epoch 22/200
5/5 [==============================] - 1s 139ms/step - loss: 0.6388 - error_in_tfi: 14.9114 - val_loss: 0.6143 - val_error_in_tfi: 14.4256
Epoch 23/200
5/5 [==============================] - 1s 140ms/step - loss: 0.5798 - error_in_tfi: 13.6147 - val_loss: 0.6044 - val_error_in_tfi: 14.2933
Epoch 24/200
5/5 [==============================] - 1s 140ms/step - loss: 0.6220 - error_in_tfi: 14.1424 - val_loss: 0.5951 - val_error_in_tfi: 14.1693
Epoch 25/200
5/5 [==============================] - 1s 137ms/step - loss: 0.6433 - error_in_tfi: 14.3848 - val_loss: 0.5874 - val_error_in_tfi: 14.0578
Epoch 26/200
5/5 [==============================] - 1s 140ms/step - loss: 0.6623 - error_in_tfi: 14.5147 - val_loss: 0.5793 - val_error_in_tfi: 13.9433
Epoch 27/200
5/5 [==============================] - 1s 138ms/step - loss: 0.6658 - error_in_tfi: 14.2131 - val_loss: 0.5697 - val_error_in_tfi: 13.8092
Epoch 28/200
5/5 [==============================] - 1s 138ms/step - loss: 0.5815 - error

Epoch 76/200
5/5 [==============================] - 1s 141ms/step - loss: 0.5861 - error_in_tfi: 13.3610 - val_loss: 0.2912 - val_error_in_tfi: 9.4808
Epoch 77/200
5/5 [==============================] - 1s 140ms/step - loss: 0.4859 - error_in_tfi: 11.7894 - val_loss: 0.2885 - val_error_in_tfi: 9.4360
Epoch 78/200
5/5 [==============================] - 1s 139ms/step - loss: 0.4737 - error_in_tfi: 12.1039 - val_loss: 0.2856 - val_error_in_tfi: 9.3950
Epoch 79/200
5/5 [==============================] - 1s 139ms/step - loss: 0.5361 - error_in_tfi: 12.9285 - val_loss: 0.2836 - val_error_in_tfi: 9.3709
Epoch 80/200
5/5 [==============================] - 1s 137ms/step - loss: 0.4674 - error_in_tfi: 11.6721 - val_loss: 0.2799 - val_error_in_tfi: 9.3130
Epoch 81/200
5/5 [==============================] - 1s 139ms/step - loss: 0.4816 - error_in_tfi: 11.3975 - val_loss: 0.2766 - val_error_in_tfi: 9.2559
Epoch 82/200
5/5 [==============================] - 1s 141ms/step - loss: 0.4717 - error_in_tf

5/5 [==============================] - 1s 139ms/step - loss: 0.5416 - error_in_tfi: 12.8699 - val_loss: 0.1705 - val_error_in_tfi: 7.4535
Epoch 131/200
5/5 [==============================] - 1s 137ms/step - loss: 0.3869 - error_in_tfi: 10.8918 - val_loss: 0.1702 - val_error_in_tfi: 7.4675
Epoch 132/200
5/5 [==============================] - 1s 138ms/step - loss: 0.3693 - error_in_tfi: 10.6831 - val_loss: 0.1693 - val_error_in_tfi: 7.4516
Epoch 133/200
5/5 [==============================] - 1s 138ms/step - loss: 0.4483 - error_in_tfi: 11.5842 - val_loss: 0.1689 - val_error_in_tfi: 7.4507
Epoch 134/200
5/5 [==============================] - 1s 138ms/step - loss: 0.4694 - error_in_tfi: 11.6155 - val_loss: 0.1674 - val_error_in_tfi: 7.4217
Epoch 135/200
5/5 [==============================] - 1s 138ms/step - loss: 0.3807 - error_in_tfi: 11.1508 - val_loss: 0.1669 - val_error_in_tfi: 7.4164
Epoch 136/200
5/5 [==============================] - 1s 138ms/step - loss: 0.4590 - error_in_tfi: 11.7

5/5 [==============================] - 1s 139ms/step - loss: 0.3472 - error_in_tfi: 10.0737 - val_loss: 0.1242 - val_error_in_tfi: 7.0056
Epoch 185/200
5/5 [==============================] - 1s 148ms/step - loss: 0.3600 - error_in_tfi: 10.7154 - val_loss: 0.1242 - val_error_in_tfi: 7.0162
Epoch 186/200
5/5 [==============================] - 1s 151ms/step - loss: 0.3391 - error_in_tfi: 10.4149 - val_loss: 0.1239 - val_error_in_tfi: 7.0171
Epoch 187/200
5/5 [==============================] - 1s 161ms/step - loss: 0.3855 - error_in_tfi: 10.8591 - val_loss: 0.1240 - val_error_in_tfi: 7.0349
Epoch 188/200
5/5 [==============================] - 1s 151ms/step - loss: 0.3281 - error_in_tfi: 9.7249 - val_loss: 0.1237 - val_error_in_tfi: 7.0407
Epoch 189/200
5/5 [==============================] - 1s 140ms/step - loss: 0.3699 - error_in_tfi: 10.4906 - val_loss: 0.1225 - val_error_in_tfi: 7.0201
Epoch 190/200
5/5 [==============================] - 1s 142ms/step - loss: 0.3098 - error_in_tfi: 9.778

                                                                                                  
 layer_normalization_4 (LayerNo  (None, 4, 1)        2           ['dropout_4[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 tf.__operators__.add_4 (TFOpLa  (None, 4, 1)        0           ['layer_normalization_4[0][0]',  
 mbda)                                                            'tf.__operators__.add_3[0][0]'] 
                                                                                                  
 conv1d_4 (Conv1D)              (None, 4, 4)         8           ['tf.__operators__.add_4[0][0]'] 
                                                                                                  
 dropout_5 (Dropout)            (None, 4, 4)         0           ['conv1d_4[0][0]']               
          

In [115]:
print("US_error")
print("--------")
print(US_error_scores)
US_error=np.mean(US_error_scores)
print("\n")
print(US_error)
print("\n")
print("WN_error")
print("--------")
print(WN_error_scores)
WN_error=np.mean(WN_error_scores)
print("\n")
print(WN_error)

US_error
--------
[16.647953033447266, 16.048969268798828, 23.443729400634766, 12.138420104980469, 14.426615715026855]


16.541137504577637


WN_error
--------
[20.61210823059082, 11.511922836303711, 14.087471008300781, 26.32403564453125, 18.58338165283203]


18.223783874511717
